In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
import os 
from tensorflow.keras.optimizers import RMSprop
import zipfile
import random
import shutil
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [3]:
train_local_zip = '/kaggle/input/dogs-vs-cats/train.zip'
test_local_zip = '/kaggle/input/dogs-vs-cats/test1.zip'

train_zip_ref = zipfile.ZipFile(train_local_zip, 'r')
test_zip_ref = zipfile.ZipFile(test_local_zip, 'r')

train_zip_ref.extractall('/kaggle/working')
test_zip_ref.extractall('/kaggle/working')

train_zip_ref.close()
test_zip_ref.close()

In [4]:
source_path = '/kaggle/working'

train_source_path = os.path.join(source_path, 'train')
test_source_path = os.path.join(source_path, 'test1')

#delete non-image files
!find /'/kaggle/working/train' -type f ! -name "*.jpg" -exec rm {} +
!find /'/kaggle/working/test1' -type f ! -name "*.jpg" -exec rm {} +

# os.listdir returns a list containing all files under the given path
print(f'there are {len(os.listdir(train_source_path))} images of train.')
print(f'there are {len(os.listdir(test_source_path))} images of test.')

there are 25000 images of train.
there are 12500 images of test.


In [5]:
root_dir = os.path.join(source_path, 'cats-v-dogs')

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

#Create Directory
os.mkdir(root_dir)

# GRADED FUNCTION: create_train_val_dirs
def create_train_val_dirs(root_path):
    
    """
    Creates directories for the train and test sets

    Args:
    root_path (string) - the base directory path to create subdirectories from

    Returns:
    None
    """
    train_path = os.path.join(root_path, 'training')
    os.mkdir(train_path)
    val_path = os.path.join(root_path, 'validation')
    os.mkdir(val_path)
    sub_train_path = os.path.join(train_path, 'cats')
    os.mkdir(sub_train_path)
    sub_train_path = os.path.join(train_path, 'dogs')
    os.mkdir(sub_train_path)
    sub_val_path = os.path.join(val_path, 'cats')
    os.mkdir(sub_val_path)
    sub_val_path = os.path.join(val_path, 'dogs')
    os.mkdir(sub_val_path)
    pass

try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [6]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/kaggle/working/cats-v-dogs/validation
/kaggle/working/cats-v-dogs/training
/kaggle/working/cats-v-dogs/validation/cats
/kaggle/working/cats-v-dogs/validation/dogs
/kaggle/working/cats-v-dogs/training/cats
/kaggle/working/cats-v-dogs/training/dogs


In [7]:
def split_data(SOURCE_DIR, SPLIT_SIZE):
    """
    Splits the data into train and test sets

    Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training

    Returns:
    None
    """

    ### START CODE HERE
    data_list = os.listdir(SOURCE_DIR)
    
    #cat_list = list(filter(lambda x:x.beginwith('cat'), data_list))
    
    train_rand = random.sample(data_list, int(len(data_list)*SPLIT_SIZE))
    
    for i in data_list:
        if os.path.getsize(os.path.join(SOURCE_DIR, i)) != 0:
            if i in train_rand:
                if i.startswith('cat'):
                    shutil.copy(os.path.join(SOURCE_DIR, i), '/kaggle/working/cats-v-dogs/training/cats')
                else:
                    shutil.copy(os.path.join(SOURCE_DIR, i), '/kaggle/working/cats-v-dogs/training/dogs')
            else:
                if i.startswith('cat'):
                    shutil.copy(os.path.join(SOURCE_DIR, i), '/kaggle/working/cats-v-dogs/validation/cats')
                else:
                    shutil.copy(os.path.join(SOURCE_DIR, i), '/kaggle/working/cats-v-dogs/validation/dogs')
        else:
            print(i, "  is zero length, so ignoring.")
    pass

In [8]:
SOURCE_DIR = train_source_path

split_size = .9

split_data(SOURCE_DIR, split_size)

print(f"\n\nOriginal cat's directory has {len(os.listdir(SOURCE_DIR))} images")

# Training and validation splits
print(f"There are {len(os.listdir('/kaggle/working/cats-v-dogs/training/cats'))} images of cats for training")
print(f"There are {len(os.listdir('/kaggle/working/cats-v-dogs/training/dogs'))} images of dogs for training")
print(f"There are {len(os.listdir('/kaggle/working/cats-v-dogs/validation/cats'))} images of cats for validation")
print(f"There are {len(os.listdir('/kaggle/working/cats-v-dogs/validation/dogs'))} images of dogs for validation")



Original cat's directory has 25000 images
There are 11267 images of cats for training
There are 11233 images of dogs for training
There are 1233 images of cats for validation
There are 1267 images of dogs for validation


In [9]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators

  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images

  Returns:
    train_generator, validation_generator - tuple containing the generators
  """
  
  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  
  return train_generator, validation_generator

In [10]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16,(3,3), activation='relu',input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  model.compile(optimizer=RMSprop(lr=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [11]:
train_generator, validation_generator = train_val_generators('/kaggle/working/cats-v-dogs/training/', '/kaggle/working/cats-v-dogs/validation/')

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

2023-12-28 02:53:07.997201: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997317: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997425: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997493: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997552: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997775: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-28 02:53:07.997846: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

Epoch 1/15
1125/1125 [==============================] - 94s 83ms/step - loss: 0.5699 - accuracy: 0.6926 - val_loss: 0.4988 - val_accuracy: 0.7752
Epoch 2/15
 247/1125 [=====>........................] - ETA: 1:07 - loss: 0.4747 - accuracy: 0.7672